In [1]:
import math as m
import numpy as np
import matplotlib as mpl  
import pandas as pd
import csv
from cvxopt import matrix, solvers

In [3]:
opened_file = open('Waypoints.csv')
from csv import reader
read_file = reader(opened_file)
wayp = list(read_file)
n=len(wayp)
waypoints = [list(map(float,i)) for i in wayp]
waypoints=np.array(waypoints)

# Define timestep
#TS = 0.2
heading = np.arctan((waypoints[1,1]-waypoints[0,1]) / (waypoints[1,0] - waypoints[0,0]))
                
#define end target
final = waypoints[-1,:]

#set initial localisation
#need to figure out the transpose thing
z0=np.array([waypoints[0,0],waypoints[0,1], 0, heading])
z = z0
z_list = z
u_list=[]
#set reference velocity
v_ref = 5

# Define horizon
#N = 10
i = 0

ctrl=[0,0]
print(z0)

[-126.05        -18.42          0.            0.15293833]


In [47]:
#probably don't need this function (using linearised dynamics for MPC)


def bikeFE(z, u):

    TS = 0.2
    lf = 1.738
    lr = 1.738
    zp = 4*[0]

    zp[2] = z[2]+TS*z[3]*np.sin(z[4]+u[2]);
    zp[3] = z[3]+TS*u[1];
    zp[4] = z[4]+TS*z[3]*np.sin(u[2])/lr;
    
    return zp

In [46]:
import osqp
import numpy as np
import scipy as sp
from scipy import sparse

def solveCFTOC(z0, goal, ctrl):
    z = z0
    u=ctrl

    lr=1.738
    TS=0.2
    # Discrete time model of a bicycle
    Ad = sparse.csc_matrix([
      [1.0,0.0,TS,0.0 ],
      [0.,1.0,(z[3]+u[1])*TS,z[2]*TS],
      [0.0,0.0,1.0,0.0],
      [0.0,0.0,TS*u[1]/lr,1.0]
    ])
    Bd = sparse.csc_matrix([
      [0.0,0.0],
      [0.0,z[2]*TS],
      [TS,0.0],
      [0.0,z[2]*TS/lr],
    ])
    [nx, nu] = Bd.shape
    # Constraints
    u0 = 0
    umin = np.array([-3,  -np.pi/4]) - u0
    umax = np.array([3,  np.pi/4]) - u0
    zmin = np.array([-1500.0, -1500.0, -2.0,  -2*np.pi])
    zmax = np.array([ 1000.0,  1000.0,  8.0,  2*np.pi])

    # Objective function
    Q = sparse.diags([5.,5.,1.,0.])
    QN = Q
    R = 0.*sparse.eye(2)
    # Initial and reference states
    z0 = z
    zr = goal

    # Prediction horizon
    N = 10

    # Cast MPC problem to a QP: x = (x(0),x(1),...,x(N),u(0),...,u(N-1))
    # - quadratic objective
    P = sparse.block_diag([sparse.kron(sparse.eye(N), Q), QN,
                           sparse.kron(sparse.eye(N), R)], format='csc')
    # - linear objective
    q = np.hstack([np.kron(np.ones(N), -Q.dot(zr)), -QN.dot(zr),
                   np.zeros(N*nu)])
    # - linear dynamics
    Ax = sparse.kron(sparse.eye(N+1),-sparse.eye(nx)) + sparse.kron(sparse.eye(N+1, k=-1), Ad)
    Bu = sparse.kron(sparse.vstack([sparse.csc_matrix((1, N)), sparse.eye(N)]), Bd)
    Aeq = sparse.hstack([Ax, Bu])
    leq = np.hstack([-z0, np.zeros(N*nx)])
    ueq = leq
    # - input and state constraints
    Aineq = sparse.eye((N+1)*nx + N*nu)
    lineq = np.hstack([np.kron(np.ones(N+1), zmin), np.kron(np.ones(N), umin)])
    uineq = np.hstack([np.kron(np.ones(N+1), zmax), np.kron(np.ones(N), umax)])
    # - OSQP constraints
    A = sparse.vstack([Aeq, Aineq], format='csc')
    l = np.hstack([leq, lineq])
    u = np.hstack([ueq, uineq])
    # Create an OSQP object
    prob = osqp.OSQP()

    # Setup workspace
    prob.setup(P, q, A, l, u, warm_start=True)

    # Simulate in closed loop
    nsim = 15
    for i in range(nsim):
        # Solve
        res = prob.solve()

        # Check solver status
        if res.info.status != 'solved':
            raise ValueError('OSQP did not solve the problem!')

        # Apply first control input to the plant
        ctrl = res.x[-N*nu:-(N-1)*nu]
        z0 = Ad.dot(z0) + Bd.dot(ctrl)

        # Update initial state
        l[:nx] = -z0
        u[:nx] = -z0
        prob.update(l=l, u=u)
        return ctrl,z0


In [49]:

for i in range(10):
    #np.linag.norm giving wrong answer
    current_dis=np.ndarray(shape=(len(waypoints),2), dtype=float)
    for k in range(len(waypoints)):
        current_dis[k,:]=waypoints[k,:]-z[:2]
        current_dis[k,0]=abs(current_dis[k,0])
        current_dis[k,1]=abs(current_dis[k,1])
    minimum_x=min(current_dis[:,0])
    minimum_y=min(current_dis[:,1])
    for val in range(len(current_dis)):
        if (current_dis[val,0] - minimum_x) <0.1 and (current_dis[val,1] - minimum_y)<0.1:
            current_idx=val
            goal_idx    = current_idx + 2   

    # Define input constraints
    #umax = np.array([3,  m.pi/4])
    #umin = np.array([-3, -m.pi/4])
    # Define goal state constraints (X,Y,V,Heading)
    heading = np.arctan((waypoints[goal_idx,1]-z[1]) / (waypoints[goal_idx,0] - z[0]))
    goal = np.array([waypoints[goal_idx, 0],waypoints[goal_idx, 1],v_ref,heading])
    print(['Goal Index:', str(goal_idx)])

    # Define constraints
    #zMax = np.array([ 1000.0,  1000.0,  8.0,  2*m.pi])
    #zMin = np.array([-1500.0, -1500.0, -2.0,  -2*m.pi])

    print(['Currently Solving for iter:', str(i)])
    solveCFTOC(z, goal,ctrl)   
    z_list = [z_list, z0]
    u_list = [u_list, ctrl]
    z=z0
    i = i + 1
print(z_list)
print(u_list)

['Goal Index:', '2']
['Currently Solving for iter:', '0']
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 64, constraints m = 108
          nnz(P) + nnz(A) = 211
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -2.2988e+04   1.26e+02   1.12e+07   1.00e-01   7.01e-04s
  50  -3.9888e+05   8.98e-04   1.34e-01   1.48e-02 